# Custom Chatbot Project

This notebook presents a Retrieval-Augmented Generation (RAG) system for the fashion domain. It uses a fashion dataset to retrieve the most relevant information for a query and feeds it as context to a GPT-based model. This approach enhances response quality by combining precise retrieval with fluent generation.

## Data Wrangling


In [62]:
import pandas as pd
import openai
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, get_embedding


In [29]:
df= pd.read_csv("data/2023_fashion_trends.csv")
df.columns

Index(['URL', 'Trends', 'Source'], dtype='object')

In [30]:
df=df.rename(columns={"Trends":"text"}).drop(['Source','URL'],axis=1)

## Custom Query Completion



In [66]:
prompt_template = """
Answer the question based on the context below, and if the 
question can't be answered based on the context, say 
"I don't know"

Context: 

{}

---

Question: {}
Answer:"""

In [79]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"
token_limit=150
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )

    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

In [80]:
# Add embeddings list to dataframe
df["embeddings"] = embeddings

In [81]:
def calculate_similarity(query,data):
    return distances_from_embeddings(query,data,distance_metric='cosine')

In [100]:
def query_openai(query,df,k=3,token_limit=token_limit):
    query_embedding=get_embedding(query, engine=EMBEDDING_MODEL_NAME)
    similaritites=calculate_similarity(query_embedding,df['embeddings'].values)
    top_k_candidates=df['text'].iloc[np.argsort(similaritites)][:k].to_list()
    token_limit_copy=token_limit
    token_limit_copy-=len(prompt_template)+len(query)
    for i,cand in enumerate(top_k_candidates):
        if token_limit_copy-len(cand)>=0:
            token_limit_copy-=len(cand)
        else:
            break
    final_prompt=prompt_template.format("\n\n###\n\n".join(top_k_candidates),query)
    response=openai.Completion.create(model=COMPLETION_MODEL_NAME,
            prompt=final_prompt,
            max_tokens=token_limit)
    return response["choices"][0]["text"].strip()

## Custom Performance Demonstration


### Question 1

In [105]:
Question1="Why did maxi skirts become a popular fashion trend in 2023?"

In [106]:
answer1=openai.Completion.create(model=COMPLETION_MODEL_NAME,
            prompt=Question1,
            max_tokens=token_limit)["choices"][0]["text"].strip()
answer1

'There are a few reasons why maxi skirts became a popular fashion trend in 2023:\n\n1. Comfort and Versatility: Maxi skirts are known for being comfortable and easy to wear, making them a go-to choice for many people. They are also versatile and can be dressed up or down for different occasions.\n\n2. Sustainable Fashion: As people become more conscious about the environmental impact of fast fashion, maxi skirts offer a more sustainable option. They can be worn for multiple seasons and can be easily styled in different ways, reducing the need to constantly buy new clothing.\n\n3. Bohemian Influence: The bohemian style, known for its flowy and feminine aesthetic, has been gaining popularity in recent years. Maxi skirts, with'

In [107]:
answer2=query_openai(Question1,df)
answer2

/tmp/ipykernel_13/937767380.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  top_k_candidates=df['text'].iloc[np.argsort(similaritites)][:k].to_list()


'The context states that maxi skirts became popular in response to the impracticality of mini skirts in 2022 and that they fit in with the cargo and denim trends. They are also described as a versatile and transitional wardrobe staple, making them popular with the rise of Y2K fashion. In addition, they were featured in the spring 2023 shows at New York Fashion Week and were praised by a Vogue Runway coordinator for their effortless and casual style. Based on this information, maxi skirts likely became popular due to their practicality, versatility, and popularity within the fashion industry.'

### Question 2

In [110]:
Question2="Why did moto detailing become a key fashion trend in 2023?"

In [112]:
answer1=openai.Completion.create(model=COMPLETION_MODEL_NAME,
            prompt=Question2,
            max_tokens=token_limit)["choices"][0]["text"].strip()
answer1

'1. Rising Interest in Adventure and Sports: Moto detailing is heavily inspired by motorcycle culture and the adventurous lifestyle associated with it. As more people are becoming interested in exploring different outdoor activities and seeking new experiences, the fashion industry has shifted its focus towards incorporating elements of adventure and sports into clothing and accessories. Thus, moto detailing has become a key trend as it embodies this sense of adventure and adds a rugged, edgy touch to fashion.\n\n2. Influence of Celebrity Culture: Celebrities and influencers play a significant role in shaping fashion trends. Many celebrities, such as musicians and actors, are often seen wearing moto-inspired outfits in their daily lives and on the red carpet. This has contributed to the popularization of moto detailing and has made it a staple'

In [113]:
answer2=query_openai(Question2,df)
answer2

/tmp/ipykernel_13/937767380.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  top_k_candidates=df['text'].iloc[np.argsort(similaritites)][:k].to_list()


'It became a key fashion trend because of the influence of motocross, which is seen in racing jackets, mixed leather, and buckle details on the runways.'